In [1]:
import os
from pkg import u
from pkg import m
import torch
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
clen_values, photon_energy_values = [1.5, 2.5, 3.5], [6000, 7000, 8000]
param_matrix = u.parameter_matrix(clen_values, photon_energy_values)

# parameters
dataset = '01'
threshold = 1
clen = 1.5 # meters 
photon_energy = 6000 # eV/ 6 keV

# instances
pm = u.PathManager()
peak_paths, water_peak_paths, labels, water_background_path = pm.select_dataset('01')
p = u.Processor(paths=pm, dataset=dataset)
dm = u.DatasetManager(paths=pm, dataset=dataset, transform=None)


true parameter matrix...

Found water background image: water01.h5
Number of peak images: 120
Number of water images: 10
Number of label images: 10
Check: Path to water background image: c:\Users\eseveret\Desktop\cxls_hitfinder\images\water\01\water01.h5



In [3]:
# generate peak/overlay/label and upate attributes
# overlaying the correct water01.h5 path to the peak images
# p.process_directory(dataset=dataset, clen=clen, photon_energy=photon_energy)

# peak, label, overlay, background are valid types
u.check_attributes(paths=pm, dataset=dataset, type='label') 

train_loader, test_loader = u.prepare(data_manager=dm, batch_size=10)


true parameter matrix...

All files in dataset 01 of type 'label' have matching attributes.

Data prepared.
Train size: 96
Test size: 24
Batch size: 10
Number of batches in train_loader: 10 



In [4]:
cfg = {
    'batch_size': 50,
    'threshold_value': 10,
    'num_epochs': 10,
    'learning_rate': 0.001,
    'num_classes': 2
}

model = m.BasicCNN1().to(device)

# loss function/ combines a Sigmoid layer and the BCELoss in one single class
criterion = nn.BCEWithLogitsLoss()
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=cfg['learning_rate'])
print("Criterion: ", criterion)
print("Optimizer: ", optimizer)
print("Learning rate: ", optimizer.param_groups[0]['lr'])
# data loaders 

loader = [train_loader, test_loader]
N = [len(loader[0].dataset), len(loader[1].dataset)]


model_fun = u.TrainTestModels(model, loader, criterion, optimizer, device, cfg)
model_fun.test_model_no_freeze()


model.cpu()  # Move model back to CPU to free up GPU memory
torch.cuda.empty_cache()  # Clear unused memory

Criterion:  BCEWithLogitsLoss()
Optimizer:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)
Learning rate:  0.001
Model training: BasicCNN1
-- epoch 0


OSError: Unable to open file (file signature not found)